# Construction Loan Estimator

## Parameters

In [1]:
# All imports
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

# Global options
pd.set_option('display.max_columns', None)
output_notebook()

Loading BokehJS ...

In [7]:
# Set scenario parameters

# Initial Costs
property_cost = 139500
const_cost = 150000
down_payment = 40000
built_value = 400000
const_close_cost = 0.05
purch_cost = 400000
purch_close_cost = 0.05

# Terms
const_loan_years = 15
const_mos = 12
years_to_consider = 30
purch_loan_years = 15

# Ongoing costs
maint_per_sqft = 0.75
assess_rate = 0.07
mill_levy = 95
const_init_hoa = 2
purch_initi_hoa = 150

# Rent costs
initial_rent = 1500

# Economic factors
inflation = 0.02
rent_growth_rate = 0.035
const_apprec = 0.057
const_loan_rate = 0.04
invest_rate = 0.0992
purch_apprec = 0.057
purch_loan_rate = 0.04

# Property estimates
square_footage = 1500

# Conversions
tot_const_loan_amt = property_cost + const_cost - down_payment
const_loan_rate_mon = const_loan_rate / 12
invest_rate_mon = invest_rate / 12
rent_growth_rate_monthly = rent_growth_rate / 12
const_apprec_mon = const_apprec / 12
current_month = datetime.date.today().replace(day=1)
const_loan_mos = const_loan_years * 12
months_to_consider = years_to_consider * 12
inflation_mon = inflation / 12
purch_loan_mos = purch_loan_years * 12
purch_loan_rate_mon = purch_loan_rate / 12
purch_apprec_mon = purch_apprec / 12
tot_purch_loan_amt = purch_cost - down_payment

In [8]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, months_to_consider + 2)
monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12)
monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Overall Month'], 0)
monthly_data['Construction Loan Month'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), monthly_data['Overall Month'] - const_mos, 0)
monthly_data['Purchase Loan Month'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, monthly_data['Overall Month'], 0)

# Calculate home value
monthly_data['Build Home Value'] = np.where(monthly_data['Overall Month'] <= const_mos, 0, built_value * (e**(const_apprec_mon * (monthly_data['Overall Month'] - const_mos))))

monthly_data['Purchase Home Value'] = purch_cost * (e**(purch_apprec_mon * (monthly_data['Overall Month'])))

# Calculate loan value
monthly_data['Construction Loan Value'] = np.where(monthly_data['Construction Loan Month'] != 0, tot_const_loan_amt * (((1 + const_loan_rate_mon) ** (const_loan_mos)) - ((1 + const_loan_rate_mon)**(monthly_data['Construction Loan Month'] - 1))) / ((1 + const_loan_rate_mon)**(const_loan_mos) - 1), tot_const_loan_amt)
monthly_data['Construction Loan Value'] = np.where(monthly_data['Overall Month'] > (const_mos + const_loan_mos), 0, monthly_data['Construction Loan Value'])

monthly_data['Purchase Loan Value'] = np.where(monthly_data['Purchase Loan Month'] != 0, tot_purch_loan_amt * (((1 + purch_loan_rate_mon) ** (purch_loan_mos)) - ((1 + purch_loan_rate_mon)**(monthly_data['Purchase Loan Month'] - 1))) / ((1 + purch_loan_rate_mon)**(purch_loan_mos) - 1), tot_purch_loan_amt)
monthly_data['Purchase Loan Value'] = np.where(monthly_data['Overall Month'] > purch_loan_mos, 0, monthly_data['Purchase Loan Value'])

# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= const_mos, tot_const_loan_amt / const_mos, 0)

# Calculate monthly payments
monthly_data['Construction Mortgage Payment'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), tot_const_loan_amt * (const_loan_rate_mon * ((1 + const_loan_rate_mon)**const_loan_mos)) / ((1 + const_loan_rate_mon)**const_loan_mos - 1), 0)

monthly_data['Purchase Mortgage Payment'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, tot_purch_loan_amt * (purch_loan_rate_mon * ((1 + purch_loan_rate_mon)**purch_loan_mos)) / ((1 + purch_loan_rate_mon)**purch_loan_mos - 1), 0)

# Calculate interest payment
monthly_data['Construction Interest Payment'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Construction Disbursements'].cumsum() * const_loan_rate_mon, monthly_data['Construction Loan Value'] * const_loan_rate_mon)
monthly_data['Construction Cumulative Interest Payments'] = monthly_data['Construction Interest Payment'].cumsum()

monthly_data['Purchase Interest Payment'] = monthly_data['Purchase Loan Value'] * purch_loan_rate_mon
monthly_data['Purchase Cumulative Interest Payments'] = monthly_data['Purchase Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Construction Principal Payment'] = np.where(monthly_data['Overall Month'] > const_mos, monthly_data['Construction Mortgage Payment'] - monthly_data['Construction Interest Payment'], 0)
monthly_data['Construction Cumulative Principal Payments'] = monthly_data['Construction Principal Payment'].cumsum()

monthly_data['Purchase Principal Payment'] = monthly_data['Purchase Mortgage Payment'] - monthly_data['Purchase Interest Payment']
monthly_data['Purchase Cumulative Principal Payments'] = monthly_data['Purchase Principal Payment'].cumsum()

# Calculate property taxes
monthly_data['Construction Property Tax Payment'] = np.where(monthly_data['Overall Month'] > const_mos, (((monthly_data['Build Home Value'] * assess_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Construction Cumulative Property Taxes'] = monthly_data['Construction Property Tax Payment'].cumsum()

monthly_data['Purchase Property Tax Payment'] = (((monthly_data['Purchase Home Value'] * assess_rate) * (mill_levy / 1000)) / 12)
monthly_data['Purchase Cumulative Property Taxes'] = monthly_data['Purchase Property Tax Payment'].cumsum()

# Calculate maintenance costs
monthly_data['Construction Maintenance Costs'] = np.where(monthly_data['Overall Month'] > const_mos, ((square_footage * maint_per_sqft) / 12), 0)
monthly_data['Construction Cumulative Maintenance Costs'] = monthly_data['Construction Maintenance Costs'].cumsum()

monthly_data['Purchase Maintenance Costs'] = ((square_footage * maint_per_sqft) / 12)
monthly_data['Purchase Cumulative Maintenance Costs'] = monthly_data['Purchase Maintenance Costs'].cumsum()

# Calculate HOA costs
monthly_data['Construction HOA Costs'] = const_init_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Construction Cumulative HOA Costs'] = monthly_data['Construction HOA Costs'].cumsum()

monthly_data['Purchase HOA Costs'] = purch_initi_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Purchase Cumulative HOA Costs'] = monthly_data['Purchase HOA Costs'].cumsum()

# Calculate total home payments
monthly_data['Construction Total Home Payments'] = monthly_data['Construction Interest Payment'] + monthly_data['Construction Principal Payment'] + monthly_data['Construction Property Tax Payment'] + monthly_data['Construction Maintenance Costs'] + monthly_data['Construction HOA Costs']
monthly_data['Construction Cumulative Home Payments'] = monthly_data['Construction Total Home Payments'].cumsum() + down_payment + (const_close_cost * tot_const_loan_amt)

monthly_data['Purchase Total Home Payments'] = monthly_data['Purchase Interest Payment'] + monthly_data['Purchase Principal Payment'] + monthly_data['Purchase Property Tax Payment'] + monthly_data['Purchase Maintenance Costs'] + monthly_data['Purchase HOA Costs']
monthly_data['Purchase Cumulative Home Payments'] = monthly_data['Purchase Total Home Payments'].cumsum() + down_payment + (purch_close_cost * tot_purch_loan_amt)

# Calculate equity metrics
monthly_data['Construction Equity Ownership Proportion'] = 1- (monthly_data['Construction Loan Value'] / tot_const_loan_amt)
monthly_data['Construction Equity Value'] = monthly_data['Construction Equity Ownership Proportion'] * monthly_data['Build Home Value']
monthly_data['Construction Equity Gain'] = monthly_data['Construction Equity Value'] - monthly_data['Construction Cumulative Home Payments']
monthly_data['Construction Gains upon Sale'] = monthly_data['Build Home Value'] - monthly_data['Construction Cumulative Home Payments'] - monthly_data['Construction Loan Value']

monthly_data['Purchase Equity Ownership Proportion'] = 1- (monthly_data['Purchase Loan Value'] / tot_purch_loan_amt)
monthly_data['Purchase Equity Value'] = monthly_data['Purchase Equity Ownership Proportion'] * monthly_data['Purchase Home Value']
monthly_data['Purchase Equity Gain'] = monthly_data['Purchase Equity Value'] - monthly_data['Purchase Cumulative Home Payments']
monthly_data['Purchase Gains upon Sale'] = monthly_data['Purchase Home Value'] - monthly_data['Purchase Cumulative Home Payments'] - monthly_data['Purchase Loan Value']

# Calculate rent payments
monthly_data['Rent Payments'] = np.where(monthly_data['Overall Month'] == 1, (initial_rent + down_payment), initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1))))
monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

# Calculate rent investment value
monthly_data['Rent Investment'] = down_payment * (e**(invest_rate_mon * (monthly_data['Overall Month'] - 1)))

# Calculate when intersection points
# monthly_data['Decision'] = np.where(monthly_data['Cumulative Rent Payments'] - monthly_data['Cumulative Home Payments'] > 0, 'Buy', 'Rent')

monthly_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,4.019045e+05,249500.0,360000.000000,20791.666667,0.0,2662.876532,69.305556,69.305556,1200.000000,1200.000000,0.0,0.000000,1462.876532,1462.876532,0.000000,0.000000,222.722088,222.722088,0.00,0.00,93.75,93.75,2.000000,2.000000,150.000000,150.000000,71.305556,52546.305556,3129.348620,61129.348620,0.0,0.000000e+00,-5.254631e+04,-3.020463e+05,0.000000,0.000000e+00,-6.112935e+04,-1.922483e+04,41500.000000,41500.000000,40000.000000
1,2,0,0,2,0,2,0.000000e+00,4.038181e+05,249500.0,358537.123468,20791.666667,0.0,2662.876532,138.611111,207.916667,1195.123745,2395.123745,0.0,0.000000,1467.752787,2930.629319,0.000000,0.000000,223.782534,446.504622,0.00,0.00,93.75,187.50,2.003336,4.003336,150.250208,300.250208,140.614447,52686.920003,3130.659275,64260.007895,0.0,0.000000e+00,-5.268692e+04,-3.021869e+05,0.004064,1.640933e+03,-6.261907e+04,-1.897902e+04,1504.381386,43004.381386,40332.037196
2,3,0,0,3,0,3,0.000000e+00,4.057408e+05,249500.0,357069.370681,20791.666667,0.0,2662.876532,207.916667,415.833333,1190.231236,3585.354980,0.0,0.000000,1472.645297,4403.274616,0.000000,0.000000,224.848030,671.352652,0.00,0.00,93.75,281.25,2.006678,6.010014,150.500834,450.751043,209.923344,52896.843347,3131.975396,67391.983292,0.0,0.000000e+00,-5.289684e+04,-3.023968e+05,0.008141,3.302989e+03,-6.408899e+04,-1.872055e+04,1508.775571,44513.156957,40666.830610
3,4,0,0,4,0,4,0.000000e+00,4.076727e+05,249500.0,355596.725384,20791.666667,0.0,2662.876532,277.222222,693.055556,1185.322418,4770.677398,0.0,0.000000,1477.554114,5880.828730,0.000000,0.000000,225.918599,897.271251,0.00,0.00,93.75,375.00,2.010025,8.020039,150.751878,601.502921,279.232247,53176.075595,3133.297009,70525.280301,0.0,0.000000e+00,-5.317608e+04,-3.026761e+05,0.012231,4.986374e+03,-6.553891e+04,-1.844935e+04,1513.182590,46026.339547,41004.403120
4,5,0,0,5,0,5,0.000000e+00,4.096137e+05,249500.0,354119.171270,20791.666667,0.0,2662.876532,346.527778,1039.583333,1180.397238,5951.074636,0.0,0.000000,1482.479295,7363.308025,0.000000,0.000000,226.994265,1124.265516,0.00,0.00,93.75,468.75,2.013378,10.033417,151.003341,752.506262,348.541156,53524.616750,3134.624138,73659.904439,0.0,0.000000e+00,-5.352462e+04,-3.030246e+05,0.016336,6.691300e+03,-6.696860e+04,-1.816536e+04,1517.602481,47543.942028,41344.777796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,357,29,0,0,0,0,2.059492e+06,2.180293e+06,0.0,0.000000,0.000000,0.0,0.000000,0.000000,88925.216091,0.000000,120508.929029,0.0,246828.943259,0.000000,356145.970234

In [9]:
# Shrink set to year starts for simpler graphs
year_data = monthly_data.loc[monthly_data['Year Start'] == 1]

year_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,4.019045e+05,249500.000000,360000.000000,20791.666667,0.000000,2662.876532,69.305556,69.305556,1200.000000,1200.000000,0.000000,0.000000,1462.876532,1462.876532,0.000000,0.000000,222.722088,222.722088,0.00,0.00,93.75,93.75,2.000000,2.000000,150.000000,150.000000,71.305556,52546.305556,3129.348620,61129.348620,0.000000,0.000000e+00,-5.254631e+04,-3.020463e+05,0.000000,0.000000e+00,-6.112935e+04,-1.922483e+04,41500.000000,41500.000000,40000.000000
12,13,1,1,0,1,13,4.019045e+05,4.254786e+05,249500.000000,342120.045893,0.000000,1845.521374,2662.876532,831.666667,6237.500000,1140.400153,15214.964432,1013.854708,1013.854708,1522.476379,19402.430486,222.722088,222.722088,235.786033,2979.563350,93.75,93.75,93.75,1218.75,2.040403,26.261815,153.030201,1969.636124,2164.033865,60069.088611,3145.442766,98785.344392,0.000000,0.000000e+00,-6.006909e+04,9.233543e+04,0.049667,2.113205e+04,-7.765330e+04,-1.542684e+04,1553.429563,59845.435151,44171.485396
24,25,2,1,0,13,25,4.254786e+05,4.504353e+05,237108.198473,323511.634869,0.000000,1845.521374,2662.876532,790.360662,15950.648961,1078.372116,28499.043758,1055.160713,13446.962240,1584.504416,38072.869547,235.786033,2979.563350,249.616253,5898.109370,93.75,1218.75,93.75,2343.75,2.081622,51.013751,156.121616,3826.031336,2177.139028,86121.938302,3162.364402,136639.804011,0.049667,2.113205e+04,-6.498989e+04,1.022484e+05,0.101357,4.565458e+04,-9.098522e+04,-9.716094e+03,1608.762272,78844.329359,48778.003052
36,37,3,1,0,25,37,4.504353e+05,4.768560e+05,224211.535833,304145.088468,0.000000,1845.521374,2662.876532,747.371786,25157.253938,1013.816962,41022.460588,1098.149588,26386.613756,1649.059571,57503.971103,249.616253,5898.109370,264.257697,8987.845073,93.75,2343.75,93.75,3468.75,2.123673,76.265710,159.275482,5719.928218,2191.011301,112336.992773,3180.159711,174702.954982,0.101357,4.565458e+04,-6.668241e+04,1.138868e+05,0.155153,7.398541e+04,-1.007175e+05,-1.992049e+03,1666.065916,98519.958647,53864.921237
48,49,4,1,0,37,49,4.768560e+05,5.048264e+05,210789.443258,283989.519086,0.000000,1845.521374,2662.876532,702.631478,33836.677547,946.631730,52754.224361,1142.889897,39853.446639,1716.244802,77726.725716,264.257697,8987.845073,279.757946,12258.811726,93.75,3468.75,93.75,4593.75,2.166574,102.027791,162.493060,7652.084356,2205.695646,138723.747050,3198.877539,212985.596160,0.155153,7.398541e+04,-6.473833e+04,1.273428e+05,0.211140,1.065892e+05,-1.063964e+05,7.851254e+03,1725.410698,118896.428120,59482.339544
60,61,5,1,0,49,61,5.048264e+05,5.344374e+05,196820.513922,263012.780708,0.000000,1845.521374,2662.876532,656.068380,41967.441606,876.709269,63662.081909,118

In [10]:
# Plot payment comparison
payments = figure(title='Rent vs. Buy Cumulative Cost')
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Cost"
payments.line(year_data['Overall Year'], year_data['Cumulative Rent Payments'], line_color='red', legend_label='Renting')
payments.line(year_data['Overall Year'], year_data['Construction Cumulative Home Payments'], line_color='blue', legend_label='Building')
payments.line(year_data['Overall Year'], year_data['Purchase Cumulative Home Payments'], line_color='green', legend_label='Purchase')

show(payments)

In [11]:
# Plot returns comparison
returns = figure(title='Rent vs. Buy Cumulative Returns')
returns.xaxis.axis_label = "Year"
returns.yaxis.formatter.use_scientific = False
returns.yaxis.axis_label = "Returns"
returns.line(year_data['Overall Year'], year_data['Rent Investment'], line_color='red', legend_label='Renting')
returns.line(year_data['Overall Year'], year_data['Construction Equity Gain'], line_color='blue', legend_label='Construction Equity Gain')
returns.line(year_data['Overall Year'], year_data['Construction Gains upon Sale'], line_color='purple', legend_label='Construction Gains upon Sale')
returns.line(year_data['Overall Year'], year_data['Purchase Equity Gain'], line_color='green', legend_label='Purchase Equity Gain')
returns.line(year_data['Overall Year'], year_data['Purchase Gains upon Sale'], line_color='grey', legend_label='Purchase Gains upon Sale')
# Include line below if you wish to see the value of the home in relation to the equity value
# payments.line(year_data['Overall Year'], year_data['Home Value'], line_color='yellow', legend_label='Home Value')

show(returns)